<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/Notebooks/CompLingNoTrainerAllDataWeightedAndBalancedLossAndEvaluateOnlyTrainClassificationLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes

ConvAbuse is just the dictionary as a string lol

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 12.4 MB/s 
     |████████████████████████████████| 182 kB 55.0 MB/s 
     |████████████████████████████████| 7.6 MB 40.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 13.0 MB/s 
     |████████████████████████████████| 212 kB 66.4 MB/s 
     |████████████████████████████████| 115 kB 71.7 MB/s 
     |████████████████████████████████| 127 kB 67.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.5 MB/s 


In [2]:
import pandas as pd

filepaths = ["/content/ArMIS_train.json", "/content/ConvAbuse_train.json", "/content/HS-Brexit_train.json", "/content/MD-Agreement_train.json"]
dfs = [pd.read_json(filepath) for filepath in filepaths]
dfs = [df.transpose() for df in dfs]
dfs = [df.astype({"hard_label": int}, errors='raise') for df in dfs]#other way aroud (do concat first) would make more sense

df_all = pd.concat(dfs)
#df_all["soft_label"] =  df_all["soft_label"].apply(lambda x: [x["0"], x["1"]])
#should probably do that, but then evaluate script has to be adapted

# Dataset

In [3]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_all[["text", "hard_label", "soft_label"]])

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", do_lower_case=True)

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/458k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/157 [00:00<?, ?B/s]

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=240)#This is the longest that roberta tokenizer would go, dunno

tokenized_datasets = dataset.map(tokenize_function, batched=False)

  0%|          | 0/10431 [00:00<?, ?ex/s]

In [6]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)

In [7]:
tokenized_datasets.set_format("torch")#not sure if neccessary

In [8]:
from torch.utils.data import DataLoader

batch_size = 4
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=batch_size)

In [72]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", num_labels=2)

Some weights of the model checkpoint at lanwuwei/GigaBERT-v4-Arabic-and-English were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

In [73]:
for name, param in model.named_parameters():
      if name not in ["classifier.weight", "classifier.bias"]:
         param.requires_grad = False

In [74]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters())

In [75]:
from transformers import get_scheduler
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [76]:
import torch
from tqdm import tqdm_notebook
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
m = model.to(device)

In [77]:
import evaluate

#TODO something is off here, untrained model gets 0.6 im ce eval

def evaluate_f1_score_huggingface(model, eval_dataloader):
  total = 0
  f1 = 0
  f1_metric = evaluate.load("f1")
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    N = predictions.shape[0]
    total+=1#the metric already gives the average, so no + batch_size
    f1 += f1_metric.compute(predictions=predictions, references=hard_labels, average = 'micro')["f1"]#micro must be used, otherwise smth wrong is returned, TODO look into that (00 00 is undefined)

  return f1/total


#slightly different to the pytorch implementations, not sure why TODO
def evaluate_ce_score(model, eval_dataloader, epsilon = 1e-12):
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)                                      
    N = predictions.shape[0]
    total+=N#maybe should be 1 here as well
    ce += -torch.sum(soft_labels*torch.log(predictions+1e-9))
  return ce/total


def evaluate_bce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.BCELoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]["1"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    N = probabilities.shape[0]
    total +=N
    ce += criterion(probabilities[:,-1], soft_labels)
  return ce/total


def evaluate_ce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.CrossEntropyLoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)                          
    N = probabilities.shape[0]
    total+=N
    ce += criterion(probabilities, soft_labels)
  return ce/total



In [78]:
#TODO do this for each dataset independently

#weight = df_all.mean(axis=0)["hard_label"]
weight = df_all["hard_label"].mean()
weight = torch.tensor([weight, 1 - weight], device = device, dtype=torch.float32)#I think this is the right way around

In [79]:

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):
      #there is also index_level_0 in the dataset, not sure if needed
      inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]
      #soft_labels = torch.cat([soft_labels["0"].unsqueeze(-1), soft_labels["1"].unsqueeze(-1)], dim=-1).to(device)# that can probably be done more intuitively
      outputs = model(input_ids=inputs, attention_mask = attention_mask)#NllLoss
      optimizer.zero_grad()
      loss = 0
      for j in range(outputs.logits.shape[0]):
        criterion = criterion = torch.nn.CrossEntropyLoss(weight = (torch.tensor([soft_labels["0"][j], soft_labels["1"][j]], device=device) + weight) / 2)#can probably be done more efficient
        loss += criterion(outputs.logits, hard_labels)
      loss = loss/batch_size
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      

      running_loss += loss.item()
      if i % 10 == 9:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 10))
        running_loss = 0.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/2347 [00:00<?, ?it/s]

[1,    10] loss: 2.892
[1,    20] loss: 2.847
[1,    30] loss: 2.473
[1,    40] loss: 2.833
[1,    50] loss: 1.531
[1,    60] loss: 3.238
[1,    70] loss: 2.752
[1,    80] loss: 2.877
[1,    90] loss: 2.794
[1,   100] loss: 2.185
[1,   110] loss: 2.941
[1,   120] loss: 2.214
[1,   130] loss: 2.322
[1,   140] loss: 2.511
[1,   150] loss: 3.011
[1,   160] loss: 2.708
[1,   170] loss: 2.454
[1,   180] loss: 2.642
[1,   190] loss: 2.443
[1,   200] loss: 2.800
[1,   210] loss: 2.426
[1,   220] loss: 1.935
[1,   230] loss: 2.383
[1,   240] loss: 2.345
[1,   250] loss: 2.554
[1,   260] loss: 2.294
[1,   270] loss: 2.506
[1,   280] loss: 2.020
[1,   290] loss: 3.441
[1,   300] loss: 2.490
[1,   310] loss: 1.983
[1,   320] loss: 2.613
[1,   330] loss: 2.886
[1,   340] loss: 2.357
[1,   350] loss: 2.631
[1,   360] loss: 2.603
[1,   370] loss: 2.604
[1,   380] loss: 2.797
[1,   390] loss: 2.568
[1,   400] loss: 2.183
[1,   410] loss: 2.813
[1,   420] loss: 2.627
[1,   430] loss: 2.571
[1,   440] 

In [80]:
evaluate_ce_score(model, eval_dataloader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/261 [00:00<?, ?it/s]

tensor(0.5278, device='cuda:0')

In [81]:
evaluate_f1_score_huggingface(model, eval_dataloader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


  0%|          | 0/261 [00:00<?, ?it/s]

0.7567049808429118

# Evaluate
create the tsv files

In [82]:
#load dev data
#create predictions on that data
#save the data in tsv
from datasets import Dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import csv
import os


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=240)#This is the longest that roberta tokenizer would go, dunno

batch_size = 1#can also be implemented with larger batch_size oc

tokenizer = AutoTokenizer.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", do_lower_case=True)

filepaths_read = ["/content/ArMIS_dev.json", "/content/ConvAbuse_dev.json", "/content/HS-Brexit_dev.json", "/content/MD-Agreement_dev.json"]
filepaths_write = ["/content/ArMIS_results.tsv", "/content/ConvAbuse_results.tsv", "/content/HS-Brexit_results.tsv", "/content/MD-Agreement_results.tsv"]

epsilon = 1e-12


In [83]:

for filepath_read, filepath_write in zip(filepaths_read, filepaths_write):
  if os.path.exists(filepath_write):
    os.remove(filepath_write)

  df = pd.read_json(filepath_read)
  df = df.transpose()

  dataset = Dataset.from_pandas(df[["text"]])
  tokenized_dataset = dataset.map(tokenize_function, batched=False)
  tokenized_dataset.set_format("torch")

  dataloader = DataLoader(tokenized_dataset, shuffle=False, batch_size=batch_size)
  with open(filepath_write, 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
    for batch in tqdm_notebook(dataloader, 0):
      inputs, attention_mask = batch["input_ids"].to(device), batch["attention_mask"].to(device)
      with torch.no_grad():
        outputs = model(input_ids=inputs, attention_mask = attention_mask)
      logits = outputs.logits

      prediction = torch.argmax(logits, dim=-1)
      probability = torch.softmax(logits, axis=-1)
      probability = torch.clip(probability, epsilon, 1. - epsilon)
      writer.writerow([prediction[0].item(), probability[0][0].item(), probability[0][1].item()])


  0%|          | 0/141 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


  0%|          | 0/141 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?ex/s]

  0%|          | 0/812 [00:00<?, ?it/s]

  0%|          | 0/168 [00:00<?, ?ex/s]

  0%|          | 0/168 [00:00<?, ?it/s]

  0%|          | 0/1104 [00:00<?, ?ex/s]

  0%|          | 0/1104 [00:00<?, ?it/s]

In [85]:
from zipfile import ZipFile

filepath = "res.zip" 

if os.path.exists(filepath):
    os.remove(filepath)

#loop over filepath names throws an string index out of range for whatever reason(also can't use content here, not sure why)
with ZipFile(filepath, 'w') as zipObj:
  zipObj.write("MD-Agreement_results.tsv")
  zipObj.write("ArMIS_results.tsv")
  zipObj.write("HS-Brexit_results.tsv")
  zipObj.write("ConvAbuse_results.tsv")
  

In [86]:
from google.colab import files
files.download("res.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>